# 策略说明
本策略基于sklearn.neighbors.NearestNeighbors算法
1.根据指定指标，寻找历史样本中与当前样本最接近的k个样本
2.计算这k个样本的次日的涨跌幅，上涨标记1，否则为0，记为label
3.计算这k个样本的label的均值，作为当前的signal，越接近1表明上涨概率越大，通过走势判断未来上涨概率大，还是下跌概率大。
其中，signal的取值范围为[-1,1]。


# 参数说明
NearestNeighbors算法参数：
至少包含close、volume的时序数据,若不是则需要修改columns,且index为datetime升序
m:移动平均
n:样本起始范围,最低为100
k:寻找k个最近邻

In [ ]:
#导入相关库
import pandas as pd 
import numpy as np      
import matplotlib.pyplot as plt
import warnings
import backtrader as bt
import datetime
warnings.filterwarnings('ignore')
plt.rcParams['font.family']='Alibaba PuHuiTi 3.0'  
plt.rcParams['axes.unicode_minus']=False#设置字体

In [ ]:
#定义基础指标函数，以放量为案例
def Increase_volume(df_volume):
    '''
    df_volume->Series,为成交量数据
    且index为datetime
    '''
    df_vol_pct=df_volume.pct_change(1)
    return df_vol_pct

In [ ]:
#定义NearestNeighbors算法函数
from sklearn.neighbors import NearestNeighbors
def vol_NearestNeighbors(data,m:int,n:int,k:int):
    '''data:为dataframe,
    至少包含close、volume的时序数据,若不是则需要修改columns,且index为datetime升序
    m:移动平均
    n:样本起始范围,最低为100
    k:寻找k个最近邻

    '''
    data['vol_pch']=Increase_volume(data.volume)#计算放量指标
    data['vol_pch_%s'%m]=data['vol_pch'].rolling(m).mean()#计算m日均值
    data['next_pct_change']=data.close.pct_change().shift(-1)#次日涨跌幅
    data["label"] = (data.next_pct_change > 0).astype(int) *2 -1
    data = data.dropna()#删除空值
    # 创建一个空的结果序列
    result = pd.Series(dtype='float64')
    for i in range(n, len(data)):#n取值为间隔区间
        # 获取当前行的vol_pch和vol_pch_n值
        vol_pch=data.vol_pch.iloc[i]
        vol_pch_n=data['vol_pch_%s'%m].iloc[i]
        # 取出历史样本
        histvol_pch=data.vol_pch.iloc[:i]
        histvol_pch_n=data['vol_pch_%s'%m].iloc[:i]
        hist_labels = data.label.iloc[:i]

        # 将histvol_pch和histvol_pch_n值组合成一个特征向量
        features = np.column_stack((histvol_pch,histvol_pch_n))
        # 使用K最近邻算法找到历史样本中最靠近的k个样本
        nn = NearestNeighbors(n_neighbors=k)#参数设为k
        nn.fit(features)
        _, indices = nn.kneighbors([[vol_pch,vol_pch_n]])
        # 取出对应的label值，并计算均值
        closest_labels = hist_labels.iloc[indices[0]]
        mean_label = closest_labels.mean()
        print('第%s次训练,均值为%s'%(i-n,mean_label))
        # 将均值添加到结果序列中
        result = pd.concat([result, pd.Series(mean_label)], ignore_index=True)
    result.index = data.iloc[n:].index
    result = result.rolling(5).mean()

    return result
        

In [ ]:
class PandasData_more(bt.feeds.PandasData):
    lines = ('result',) # 要添加的线
    # 设置 line 在数据源上的列位置
    params=(('result', -1),)
    # -1表示自动按列明匹配数据，也可以设置为线在数据源中列的位置索引 
cerebro = bt.Cerebro()
datafeed = PandasData_more(dataname=data,
                               fromdate=datetime.datetime(2023,1,1),
                               todate=datetime.datetime(2024,2,29)) 
cerebro.adddata(datafeed, name='000300.SH') # 通过 name 实现数据集与股票的一一对应

In [ ]:
# 创建策略
class vol_NearestNeighbors_day(bt.Strategy):
    def __init__(self):
        print('回测开始')
    def next(self):
        #计算买卖条件
        #空仓时开多
        if self.getposition(self.data0).size==0:#空仓情况下
            if self.data0.result[0]>0.0: #均值加标准差
                self.order = self.order_target_percent(self.data0,0.99)
                print('时间：%s买入开仓成功'%self.datetime.date(0),self.data0.result[0])
        elif self.getposition(self.data0).size>0:#持有多单仓情况下
            if self.data0.result[0]<-0.1: 
                self.order = self.order_target_percent(self.data0,0)
                print('时间：%s卖减多仓成功'%self.datetime.date(0),self.data0.result[0])

cerebro.addstrategy(vol_NearestNeighbors_day)
# 初始资金 1,000,000
cerebro.broker.setcash(10000000.0)
# 佣金，双边各 0.0003
cerebro.broker.setcommission(commission=0.0003)
# 滑点：双边各 0.0001
cerebro.broker.set_slippage_perc(perc=0.0001)
cerebro.addanalyzer(bt.analyzers.TimeReturn,_name='_TimeReturn')
# 启动回测
result=cerebro.run()